In [1]:
import pandas as pd
from datetime import datetime as dt
import re
import numpy as np

In [2]:
jobs = pd.read_csv("../data/fullsample.csv", nrows = 10000)
display(jobs.head(3))
print(jobs.shape)

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
0,30616928,RUNNING,2021-07-31T22:15:00,Unknown,2048Mn,0,10:04:00,67-22:14:22,1,1,production,0:0
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05T11:36:32,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05T19:04:53,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0


(10000, 12)


In [3]:
jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   JOBID      10000 non-null  object
 1   STATE      10000 non-null  object
 2   BEGIN      10000 non-null  object
 3   END        10000 non-null  object
 4   REQMEM     10000 non-null  object
 5   USEDMEM    10000 non-null  object
 6   REQTIME    10000 non-null  object
 7   USEDTIME   10000 non-null  object
 8   NODES      10000 non-null  int64 
 9   CPUS       10000 non-null  int64 
 10  PARTITION  10000 non-null  object
 11  EXITCODE   10000 non-null  object
dtypes: int64(2), object(10)
memory usage: 937.6+ KB


In [4]:
jobs['END'].sort_values()

7259    2021-09-01T00:00:01
7551    2021-09-01T00:00:01
7280    2021-09-01T00:00:01
7579    2021-09-01T00:00:02
7276    2021-09-01T00:00:02
               ...         
7676                Unknown
8621                Unknown
7343                Unknown
9122                Unknown
0                   Unknown
Name: END, Length: 10000, dtype: object

In [ ]:
ce5 = pd.read_csv('../data/slurm_wrapper_ce5.log',
                  header=None,
                  delimiter=' - ',
                  engine='python',
                  # nrows=100000
                  )

In [ ]:
# set column names
colnames = ['END', 'USER', 'RETRY', 'TIME', 'RETURNCODE', 'COMMAND']
ce5.columns = colnames

# Parse 'END' column with optional milliseconds
# ce5['END'] = pd.to_datetime(ce5['END'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce') # don't like coercing errors
# ce5['END'] = [dt.strptime(row + ('.000000' if '.' not in row else ''), '%Y-%m-%d %H:%M:%S.%f') for row in ce5['END']] # works, but not as efficient

# optional milliseconds function
def parse_end_column(series):
    # format with milliseconds
    parsed = pd.to_datetime(series, format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')
    # Handle rows where the first attempt failed
    wrong = parsed.isna()
    if wrong.any():
        parsed[wrong] = pd.to_datetime(series[wrong], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    return parsed

# Apply the parsing function to the 'END' column
ce5['END'] = parse_end_column(ce5['END'])

# Clean and convert other columns
ce5['TIME'] = ce5['TIME'].str.replace('time', '', regex=False).astype(float)
ce5['USER'] = ce5['USER'].str.replace('user', '', regex=False).astype(int)
ce5['RETRY'] = ce5['RETRY'].str.replace('retry', '', regex=False).astype(int)
ce5['RETURNCODE'] = ce5['RETURNCODE'].str.replace('returncode', '', regex=False).astype(float)
ce5['COMMAND'] = ce5['COMMAND'].str.replace('command', '', regex=False)

# Display the updated DataFrame
ce5.head()

,END,USER,RETRY,TIME,RETURNCODE,COMMAND
0,2020-10-16 08:15:39.278699,0,0,0.073476,0.0,"['/usr/bin/sacct', '-u', 'appelte1', '-S', '2..."
1,2020-10-16 08:18:08.313309,0,0,0.183632,0.0,"['/usr/bin/sacct', '-u', 'appelte1', '-S', '2..."
2,2020-10-16 08:22:48.128689,0,0,0.075471,0.0,"['/usr/bin/sacct', '-u', 'appelte1', '-S', '2..."
3,2020-10-16 08:25:13.257408,0,0,0.094844,0.0,"['/usr/bin/sacct', '-u', 'appelte1', '-S', '2..."
4,2020-10-16 08:31:01.460723,0,0,0.074988,0.0,"['/usr/bin/sacct', '-u', 'appelte1', '-S', '2..."


In [ ]:
ce5['END'].sort_values().iloc[-1]

pandas._libs.tslibs.timestamps.Timestamp

In [12]:
# ce6 = pd.read_csv('../data/slurm_wrapper_ce6.log',
#                   header=None,
#                   delimiter=' - ',
#                   engine='python',
#                   nrows=100)

In [13]:
# ce6.columns=colnames
# ce6['TIME'] = [float(re.sub(r'time', '', row)) for row in ce6['TIME']]
# ce6['USER'] = [float(re.sub(r'user', '', row)) for row in ce6['USER']]
# ce6['RETRY'] = [float(re.sub(r'retry', '', row)) for row in ce6['RETRY']]
# ce6['RETURNCODE'] = [float(re.sub(r'returncode', '', row)) for row in ce6['RETURNCODE']]
# ce6['COMMAND'] = [re.sub(r'command', '', row) for row in ce6['COMMAND']]
# ce6.head()